<a href="https://colab.research.google.com/github/KristinaFateyeva/test/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%A1%D0%BE%D0%B7%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_%D0%BC%D0%B5%D1%82%D0%BE%D0%B4%D0%B8%D1%87%D0%BA%D0%B8_%D0%BF%D0%BE_%D1%81%D0%BE%D0%B7%D0%B2%D0%BE%D0%BD%D1%83_%D1%81_%D0%9E%D1%86%D0%B5%D0%BD%D1%89%D0%B8%D0%BA%D0%BE%D0%BC_(%D1%81_%D0%BF%D0%BE%D0%B4%D0%B4%D0%B5%D1%80%D0%B6%D0%BA%D0%BE%D0%B9_mp3)_05_02_2024%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Настройка окружения

!pip install faiss-cpu langchain openai tiktoken moviepy pydub nltk >/dev/null 2>&1

import os
from datetime import datetime
from openai import OpenAI
import openai
import getpass

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.docstore.document import Document
from langchain.text_splitter import NLTKTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.prompts.prompt import PromptTemplate
from langchain.callbacks import get_openai_callback
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from IPython.display import display
from IPython.display import HTML, clear_output
import tiktoken
import zipfile
import requests
from moviepy.editor import VideoFileClip
from pydub import AudioSegment
import nltk
import re
import time
import datetime
from tqdm.notebook import trange, tqdm
nltk.download('punkt', quiet=True)

print('Настройка завершилась удачно!')

In [ ]:
#@title Установка ключа OpenAI

os.environ["OPENAI_API_KEY"] = getpass.getpass("Введите OpenAI API Key и нажмите Enter:")
client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  # this is also the default, it can be omitted
)

# Функция настройки стиля для переноса текста в выводе ячеек
def set_text_wrap_css():
    css = '''
    <style>
    pre {
        white-space: pre-wrap;
    }
    </style>
    '''
    display(HTML(css))

get_ipython().events.register('pre_run_cell', set_text_wrap_css)

In [ ]:
#@title Загрузка и обработка аудио (.mp3, .m4a) по ссылке Яндекс диска
url_file = "" #@param {type:"string"}

def get_source_file(path_base):
  source_path = ''
  type_file = ''
  audio_path = ''
  full_url = f'https://getfile.dokpub.com/yandex/get/{path_base}'
  response = requests.get(full_url)
  extract_dir = '/content'
  headers_file = response.headers['Content-Disposition']

  if headers_file.endswith('.mp4'):
    type_file = 'mp4'
    source_path = '/content/source_load_file.mp4'
    with open(source_path, 'wb') as file:
      file.write(response.content)
    print(f'Загружен файл: {headers_file[-19:]}')

  elif headers_file.endswith('.m4a'):
    type_file = 'm4a'
    source_path = '/content/source_load_file.m4a'
    with open(source_path, 'wb') as file:
      file.write(response.content)
    print(f'Загружен файл: {headers_file[-19:]}')

  elif headers_file.endswith('.mp3'):
    type_file = 'mp3'
    source_path = '/content/source_load_file.mp3'
    with open(source_path, 'wb') as file:
      file.write(response.content)
    print(f'Загружен файл: {headers_file[-19:]}')

  else:
    print(f'Не поддерживаемый тип файла (поддерживается только .m4a, .mp4 или .mp3): {headers_file.split["."][-1]}')

  return source_path, type_file


def extract_audio(video_path, audio_path):
    video_clip = VideoFileClip(video_path)
    audio_clip = video_clip.audio
    audio_clip.write_audiofile(audio_path)

def convert_m4a_to_mp3(m4a_path, mp3_path):
    audio = AudioSegment.from_file(m4a_path)
    audio.export(mp3_path, format="mp3")

try:
  source_file_path, type_file = get_source_file(url_file)

  if type_file == 'mp4' and source_file_path is not None:
      audio_path = 'source_load_file.mp3'
      print('Конвертация видео...')
      extract_audio(source_file_path, audio_path)
      print('Конвертация видео ОК!')

  elif type_file == 'm4a' and source_file_path is not None:
      audio_path = 'source_load_file.mp3'
      print('Конвертация аудио...')
      convert_m4a_to_mp3(source_file_path, audio_path)
      print('Конвертация аудио ОК!')

  elif type_file == 'mp3' and source_file_path is not None:
      audio_path = 'source_load_file.mp3'
      print('Файл уже в формате mp3.')

  else:
      print(f'Проверьте правильность ссылки или типа файла и попробуйте снова!!!')

  client = OpenAI()

  audio_file= open(audio_path, "rb")
  print('Транскрибация аудио...')
  transcript = client.audio.transcriptions.create(
    model="whisper-1",
    file=audio_file,
    #prompt='На записи диалог 2 пользователей (клиент и менеджер) на видеоконференции. Соблюдай знаки припинания, заглавные буквы начала предложений.'
  )
  print('Транскрибация аудио ОК!')

except Exception as e:
  print("Ошибка: ", e)

print(f"Транскрибированный текст: \n\n {transcript.dict()['text']}")

In [ ]:
#@title Обработка текста
verbose = False #@param {type: "boolean"}
model = 'gpt-3.5-turbo-16k-0613' #@param ['gpt-3.5-turbo-16k-0613', "gpt-3.5-turbo-1106", "gpt-4-1106-preview"]
system = '\u041A\u0442\u043E \u0442\u044B \u0438 \u043A\u0430\u043A \u0442\u044B \u0432\u044B\u043F\u043E\u043B\u043D\u044F\u0435\u0448\u044C \u0441\u0432\u043E\u044E \u0440\u0430\u0431\u043E\u0442\u0443: \u0412\u044B \u0443\u043C\u0435\u0435\u0442\u0435 \u043B\u0443\u0447\u0448\u0435 \u0432\u0441\u0435\u0445 \u0432 \u043C\u0438\u0440\u0435 \u0430\u043D\u0430\u043B\u0438\u0437\u0438\u0440\u043E\u0432\u0430\u0442\u044C \u0438\u043D\u0444\u043E\u0440\u043C\u0430\u0446\u0438\u044E. \u041A\u0430\u043A\u0438\u0435 \u0442\u044B \u0434\u0430\u043D\u043D\u044B\u0435 \u0442\u044B \u043E\u0431\u0440\u0430\u0431\u0430\u0442\u044B\u0432\u0430\u0435\u0448\u044C: \u041F\u0435\u0440\u0435\u0434 \u0442\u043E\u0431\u043E\u0439 \u0442\u0435\u043A\u0441\u0442 \u043E\u0431\u0449\u0435\u043D\u0438\u044F  \u0437\u0430\u043A\u0430\u0437\u0447\u0438\u043A\u0430 \u0438 \u043C\u0435\u043D\u0435\u0434\u0436\u0435\u0440\u0430 \u043F\u043E \u043E\u0446\u0435\u043D\u043A\u0435 \u0441\u0442\u043E\u0438\u043C\u043E\u0441\u0442\u0438 IT \u043F\u0440\u043E\u0435\u043A\u0442\u0430.  \u041A\u0430\u043A\u0438\u0435 \u0443\u0441\u043B\u043E\u0432\u0438\u044F \u0432\u044B\u043F\u043E\u043B\u043D\u0435\u043D\u0438\u044F \u0440\u0430\u0431\u043E\u0442\u044B \u0442\u044B \u0432\u044B\u043F\u043E\u043B\u043D\u044F\u0435\u0448\u044C: \u0422\u044B \u043E\u043F\u0438\u0441\u044B\u0432\u0430\u0435\u0448\u044C \u0442\u0435\u043A\u0441\u0442 \u0442\u0430\u043A\u0438\u043C \u043E\u0431\u0440\u0430\u0437\u043E\u043C, \u0447\u0442\u043E\u0431\u044B \u0431\u044B\u043B\u043E \u043F\u043E\u043D\u044F\u0442\u043D\u043E \u043E \u043A\u0430\u043A\u0438\u0445 \u0442\u0435\u0445\u043D\u0438\u0447\u0435\u0441\u043A\u0438\u0445 \u0434\u0430\u043D\u043D\u044B\u0445 \u0448\u043B\u0430 \u0440\u0435\u0447\u044C \u0438 \u043E \u0447\u0435\u043C \u0431\u044B\u043B  \u0434\u0438\u0430\u043B\u043E\u0433, \u0442\u044B \u0432\u0441\u0435\u0433\u0434\u0430 \u0432\u044B\u043F\u0438\u0441\u044B\u0432\u0430\u0435\u0448\u044C \u0442\u043E\u0447\u043D\u044B\u0435 \u0442\u0435\u0445\u043D\u0438\u0447\u0435\u0441\u043A\u0438\u0435 \u0434\u0430\u043D\u043D\u044B\u0435, \u043E \u043A\u043E\u0442\u043E\u0440\u044B\u0445 \u0431\u044B\u043B\u043E \u043D\u0430\u043F\u0438\u0441\u0430\u043D\u043E \u0432 \u0438\u0441\u0445\u043E\u0434\u043D\u043E\u043C \u0442\u0435\u043A\u0441\u0442\u0435.' #@param {type:"string"}
user = "\u041F\u043E\u0436\u0430\u043B\u0443\u0439\u0441\u0442\u0430, \u0434\u0430\u0432\u0430\u0439\u0442\u0435 \u043F\u043E\u0434\u0443\u043C\u0430\u0435\u043C \u0448\u0430\u0433 \u0437\u0430 \u0448\u0430\u0433\u043E\u043C: \u043F\u043E\u0434\u0443\u043C\u0430\u0439 \u043D\u0430\u0434 \u0442\u0435\u043A\u0441\u0442\u043E\u043C, \u043D\u0430\u043F\u0438\u0448\u0438 \u043E\u0447\u0435\u043D\u044C \u043A\u0440\u0430\u0442\u043A\u043E \u0441\u0443\u0445\u043E\u0439 \u0442\u0435\u0445\u043D\u0438\u0447\u0435\u0441\u043A\u0438\u0439 \u043F\u0435\u0440\u0435\u0441\u043A\u0430\u0437 \u043E\u0431\u0449\u0435\u043D\u0438\u044F \u0441\u043F\u0438\u0441\u043A\u043E\u043C \u043F\u043E  \u0445\u043E\u0434\u0443 \u0434\u0438\u0430\u043B\u043E\u0433\u0430; \u0412\u0430\u0436\u043D\u044B\u0435 \u0442\u0435\u0445\u043D\u0438\u0447\u0435\u0441\u043A\u0438\u0435 \u0434\u0430\u043D\u043D\u044B\u0435 \u0434\u043B\u044F \u043E\u0446\u0435\u043D\u043A\u0438 \u043F\u0440\u043E\u0435\u043A\u0442\u0430: \"\u0421\u043F\u0438\u0441\u043E\u043A \u0412\u0421\u0415\u0425 \u0432\u0430\u0436\u043D\u044B\u0445 \u0442\u0435\u0445\u043D\u0438\u0447\u0435\u0441\u043A\u0438\u0445 \u0434\u0430\u043D\u043D\u044B\u0445 \u043E\u0431\u0441\u0443\u0436\u0434\u0430\u0435\u043C\u044B\u0445 \u0437\u0430\u043A\u0430\u0437\u0447\u0438\u043A\u043E\u043C \u043A\u043E\u0442\u043E\u0440\u044B\u0435 \u0442\u044B \u0440\u0430\u0441\u043F\u043E\u0437\u043D\u0430\u043B \u0432 \u0438\u0441\u0445\u043E\u0434\u043D\u043E\u043C \u0442\u0435\u043A\u0441\u0442\u0435\"." #@param {type:"string"}
temperature = 0 #@param {type: "slider", min: 0, max: 1, step:0.1}
chunk_size_copywriter = 8000 # @param {type:"slider", min:1000, max:8000, step:100}
fragment_size = 8000 # @param {type:"slider", min:4000, max:8000, step:100}
result_file_path = 'result_copywriter.txt' #@param {type:"string"}


def num_tokens_from_messages(messages, model="gpt-3.5-turbo-16k-0613"):
    """Return the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model in {
        "gpt-3.5-turbo-0613",
        "gpt-3.5-turbo-16k-0613",
        "gpt-3.5-turbo-16k",
        "gpt-4-0314",
        "gpt-4-32k-0314",
        "gpt-4-0613",
        "gpt-4-32k-0613",
        "gpt-3.5-turbo-1106",
    }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif "gpt-3.5-turbo" in model:
        print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0613.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613")
    elif "gpt-4" in model:
        print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return num_tokens_from_messages(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens."""
        )
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens


# Функция дробления текста на чанки
def split_text(text, chunk_size=chunk_size_copywriter, verbose=verbose):
    source_chunks = []
    splitter = RecursiveCharacterTextSplitter(separators=['\n', '\n\n', '. '], chunk_size=chunk_size, chunk_overlap=0)

    for chunk in splitter.split_text(text):
        source_chunks.append(Document(page_content=chunk, metadata={}))

    if verbose:
      print(f'\n\nФрагмент разбит на {len(source_chunks)} чанк[-а/-ов]:')

    return source_chunks

# Функция разбиения текста после парсинга на фрагменты
def split_parced_text(text, fragment_size=fragment_size):
    sentences = nltk.sent_tokenize(text)

    result = []
    current_segment = ""

    for sentence in sentences:
        if len(current_segment) + len(sentence) <= fragment_size:
            current_segment += sentence + " "
        else:
            result.append(current_segment)
            current_segment = sentence + " "

    if current_segment:
        result.append(current_segment)

    return result

# Функция получения ответа от копирайтера
def answer_index_copywriter(model, system, user, chunk, temperature=temperature):
    client = OpenAI()
    messages = [
        {'role': 'system', 'content': system},
        {'role': 'user', 'content': user + f'\n{chunk}'}
    ]

    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    if verbose:
      print(f'\n====================\n\n{num_tokens_from_messages(messages)} токенов будет использовано на чанк.\n\n')
    answer = completion.choices[0].message.content

    return answer

# Функция обработки одного фрагмента текста
def process_one_fragment(fragment, model=model, system=system, user=user, verbose=verbose):
    source_chunks = split_text(fragment)
    processed_text = ''

    for chunk in source_chunks:
        attempt = 0

        while attempt < 3:
            try:
                answer = answer_index_copywriter(model, system, user, chunk.page_content)
                break

            except Exception as e:
                attempt += 1
                print(f'\n\nПопытка {attempt} не удалась из-за ошибки: {str(e)}')
                time.sleep(10)
                if attempt == 3:
                    answer = ''
                    print(f'\n\nОбработка элемента не удалась после 3 попыток:\n{chunk}')

        processed_text += f'{answer}\n\n'
        if verbose:
          print (f'{answer}')

    return processed_text


# Функция обработки всего текста копирайтером
def process_data(text,
                 fragment_size=fragment_size,
                 system=system,
                 user=user,
                 result_file_path=result_file_path,
                 verbose=verbose):

    # Добавляем заголовок лендинга в результирующий текст (заголовок первого уровня)
    result_text = ''

    # Делим текст на фрагменты
    fragments = split_parced_text(text, fragment_size)
    if verbose:
      print(f'Количество фрагментов: {len(fragments)}')

    # Обрабатываем каждый фрагмент копирайтером
    for fragment in tqdm(fragments):
        processed_text = process_one_fragment(fragment)
        result_text += f'{processed_text}\n\n'

    # if verbose:
    print('\n\nВесь текст обработан!')

    return result_text


# Функция очистки текста после копирайтера
def text_cleaner(text, patterns_to_remove):
    for pattern in patterns_to_remove:
        text = re.sub(pattern, '', text)

    return text


# Функция сохранения текста в файл
def save_result(text, result_file_path=result_file_path, verbose=verbose):
    with open(result_file_path, "w", encoding="utf-8") as file:
        file.write(text)
    # if verbose:
    print(f'Результат сохранен в {result_file_path}!!!')


# Получаем сырой результат работы копирайтера
text = process_data(transcript.dict()['text'])

# Чистим клинером
patterns_to_remove = [
        r' Раздел \d+:',
        r'\+ 100% текста относящегося к этому разделу:',
        r'\(100% текста\)',
        r'Весь текст, относящийся к этому разделу:',
        r'100% текста относящегося к этому разделу:',
        r'100% текста относятся к этому разделу:',
        r'Вот так можно разбить данный текст на разделы, сохраняя весь текст на 100%.',
        r'Вот так можно разделить данный текст на разделы, сохраняя весь текст на 100%.',
        r'Весь текст, относящийся к этому разделу:',
        r'\(100% текста относящегося к этому разделу\)',
        r'Разделы в тексте и их названия:',
        r'Этот раздел содержит следующий текст:',
        r'Текст, относящийся к этому разделу:',
        r'Текст: ',
        r'## Разделы и текст:',
        r'Текст, относящийся к этому разделу:',
        r'Текст, разбитый на разделы:',
        r'Таким образом, текст разделен на соответствующие разделы, сохраняя весь текст на 100%.',
        r'100% исходного текста относящегося к этому разделу:',
        r'Название раздела: ',
        r'Раздел: ',
        r'По анализу текста, можно выделить следующие разделы:',
        r'Надеюсь, это поможет вам разделить текст на соответствующие разделы:',
        r'# Название раздела',
        r'## Название подраздела + часть исходного текста',
        r'## Название подраздела'
    ]

text = text_cleaner(text, patterns_to_remove)
print(f'Результат:\n\n {text}')

# Сохраняем результат в сессионном хранилище
save_result(text)

# # Разделяем текст на чанки (## разделы)
# chunks = [chunk.strip() for chunk in text.split('##') if chunk.strip()]

# # Создаем документы из чанков
# docs = [Document(page_content=chunk, metadata={}) for chunk in chunks]

# # Создаем индексную базу
# db = FAISS.from_documents(docs, OpenAIEmbeddings())

# # Сохранение базы в сессионное хранилище
# db.save_local('faiss_index')
# print('База сохранена в сессионное хранилище (faiss_index)')